In [1]:
%config InlineBackend.figure_format = 'retina'
%pylab inline
### Libraries 
import warnings
warnings.filterwarnings('ignore')

import ee
#earthengine authenticate
ee.Authenticate()
ee.Initialize()

Populating the interactive namespace from numpy and matplotlib


Enter verification code: 4/1AWtgzh7RIooMt4incFpuSBSreGDHeScXgZbEteMpgAgUV2flciCNelM2IPI

Successfully saved authorization token.


In [3]:
import sys
sys.path.insert(1, '../Codes/')
from geeCodes import *

In [7]:
import shapely

In [5]:
dfc = gpd.read_file('../data_revision/cities/all/gdfCities.shp')
dfc = dfc.to_crs("EPSG:4326")
cities = dfc['UC_NM_MN']
years_str = []
years = np.arange(2002,2022)
for y in years:
    years_str.append(str(y))


In [32]:
SUHI = []
for idx in range(0,200):
    city = cities[idx]
    geo = dfc[dfc['UC_NM_MN']==city].reset_index(drop=True)
    S = returnCityBoundary(geo)
    bounds = S[0]
    ####### GEE Geometry ###############################################
    geometry = returnGeometry(S[0])
    
    
    ####### Urban
    geo = dfc[dfc['UC_NM_MN']==city].reset_index(drop=True)
    
    g = [i for i in geo.geometry]

    all_coords = []
    if type(g[0].boundary)==shapely.geometry.linestring.LineString:
        coords = np.dstack(g[0].boundary.coords.xy).tolist()
        all_coords.append(*coords) 
        geometry2 = ee.Geometry.Polygon(all_coords)
    else:
        for b in g[0].boundary: # for first feature/row
            coords = np.dstack(b.coords.xy).tolist()
            all_coords.append(*coords) 
        geometry2 = ee.Geometry.Polygon(all_coords)
    
    ###### Rural 
    tmp = bounds['geometry'][0]
    tmp1 = geo['geometry'][0]
    tmp2 = tmp.difference(tmp1)
    gdftmp =  gpd.GeoDataFrame(index=[0], crs=bounds.crs, geometry=[tmp2])
    g2 = [i for i in gdftmp.geometry]
    
    
    all_coords = []
    for b in g2[0].boundary: # for first feature/row
        coords = np.dstack(b.coords.xy).tolist()
        all_coords.append(*coords) 
    geometry3 = ee.Geometry.Polygon(all_coords)
    
    
    modis = ee.ImageCollection ('MODIS/006/MOD11A1').select(['LST_Day_1km']).map(toCelciusDay)
    modis = modis.filterDate("2010-01-01", "2023-01-01")   ### Data over the years
    modis_average =  modis.reduce(ee.Reducer.mean());   
    
    
    urban = modis_average.select(['LST_Day_1km_mean']).reduceRegion(
                        reducer =  ee.Reducer.toList(),
                        geometry = geometry2,
                        scale =   926.625433056,
                        crs =  'epsg:4326')
    
    rural = modis_average.select(['LST_Day_1km_mean']).reduceRegion(
                        reducer =  ee.Reducer.toList(),
                        geometry = geometry3,
                        scale =   926.625433056,
                        crs =  'epsg:4326')
    
    urban = ee.List(urban.get('LST_Day_1km_mean')).getInfo()
    rural = ee.List(rural.get('LST_Day_1km_mean')).getInfo()
    print(np.mean(urban)-np.mean(rural), city)
    
    SUHI.append(np.mean(urban)-np.mean(rural))

2.361238793816586 Guangzhou
3.2791117497963 Los Angeles
2.9097193953852383 New York
4.631586590614319 Tokyo
5.053973988924824 Jakarta
4.670681184048984 Chicago
1.5303403604105732 Dallas
3.0481702773759167 Houston
1.2646471801955457 Shanghai
0.9158224004640907 Dhaka
5.034385660293022 Osaka [Kyoto]
3.849336857034899 Miami
0.03679229476583501 Kolkata
4.105867633844625 Nagoya
2.8736768332867904 Bangkok
3.7152466926895897 Detroit
-0.10504606996483545 Comilla
0.8714559305119813 Delhi [New Delhi]
2.808734451157129 Jieyang
1.6586106589176026 Seoul
0.5134860414512517 Phoenix
1.723945632568956 Beijing
4.040997379337959 Mexico City
6.113327121517315 Quezon City [Manila]
5.78764570109861 Toronto
5.039261846766184 São Paulo
0.29373892087114584 Buenos Aires
2.807210535307167 Suzhou
2.425410815201783 Seattle
2.6096860220997495 Moscow
1.9522104661612723 London
-0.10637286326947404 Tijuana
4.671008750177652 Surabaya
-0.42152430054218115 Johannesburg
2.218050468705183 Paris
1.4938428737438763 Melbourne


In [3]:


def hotMonths(df, year): 
    df['valid_time'] = pd.to_datetime(df['valid_time'])
    df_filtered = df[df['valid_time'].dt.strftime('%Y') == year].reset_index(drop=True)
    df_filtered = df_filtered.groupby('valid_time')['t2m'].mean().reset_index()
    df_filtered = df_filtered.sort_values('t2m', ascending=False)
    idxs = list(df_filtered.index[0:3])
    df_filtered = df_filtered.iloc[:3]
    df_filtered = df_filtered.sort_values('valid_time', ascending=True)
    months = df_filtered['valid_time']
    return idxs, months.values

def returnGEEDates(df, y):
    idxs, months = hotMonths(df, str(y))
    idxs = np.sort(idxs)+1
    s1 = "%02d" % (idxs[0],)
    s2 = "%02d" % (idxs[2],)
    ld = monthrange(y, idxs[2])
    s3 = "%02d" % (ld[1],)
    return ee.Date('%s-%s-01'%(str(y),s1)), ee.Date('%s-%s-%s'%(str(y),s2,s3))

def toCelciusDay(image):
    lst = image.select('LST_Day_1km').multiply(0.02).subtract(273.15);
    overwrite = True;
    result = image.addBands(lst, ['LST_Day_1km'], overwrite);
    return result; 

def toCelciusNight(image):
    lst = image.select('LST_Night_1km').multiply(0.02).subtract(273.15);
    overwrite = True;
    result = image.addBands(lst, ['LST_Night_1km'], overwrite);
    return result; 

def bitwiseExtract(value, fromBit, toBit):
    if not toBit in locals():
        toBit = fromBit;
    maskSize = ee.Number(1).add(toBit).subtract(fromBit);
    mask = ee.Number(1).leftShift(maskSize).subtract(1);
    return value.rightShift(fromBit).bitwiseAnd(mask);

def QC_Day_mask(image2):
    return image2.updateMask(bitwiseExtract(image2.select('QC_Day'), 0, 1));


def QC_Night_mask(image3):
    return image3.updateMask(bitwiseExtract(image3.select('QC_Night'), 0, 1));


def funcMean(image):
    img_Mean_src = image.expression(
      '(Day + Night)/2', {
      'Day': image.select('LST_Day_1km'),
      'Night': image.select('LST_Night_1km')
      }
    );
    img_Mean = img_Mean_src.rename('LST_Daily_mean_1km');
    image = image.addBands([img_Mean]);
    time = image.get('system:time_start');
    return image.set('system:time_start', time);


def funcMax(image):
    img_Day = image.select('LST_Day_1km');
    img_Night = image.select('LST_Night_1km');
    img_Max = img_Day.max(img_Night).rename('LST_Daily_max_1km');
    image = image.addBands([img_Max]);
    time = image.get('system:time_start');
    return image.set('system:time_start', time);

def funcMin(image):
    img_Day = image.select('LST_Day_1km');
    img_Night = image.select('LST_Night_1km');
    img_Min = img_Day.min(img_Night).rename('LST_Daily_min_1km');
    image = image.addBands([img_Min]);
    time = image.get('system:time_start');
    return image.set('system:time_start', time);

def hotdays(image):
    hot = image.gt(35);
    return image.addBands(hot.rename('hotdays').set('system:time_start', image.get('system:time_start')));

def hotdays1(image):
    hot = image.gt(lst90);
    return image.addBands(hot.rename('hotdays').set('system:time_start', image.get('system:time_start')));


def consecutiveDays(this_img, cum_prev_max):
  ## Load cumulative # days
    cum_img = ee.Image(cum_prev_max).select(0); 
  ## Load previous day's image
    prev_img = ee.Image(cum_prev_max).select(1);
  ## Load maximum # consecutive data
    max_run = ee.Image(cum_prev_max).select(2); 
  ## Set masked pixels to 0
    this_img = this_img.unmask();
  ## If this and previous day were > 35, record this consecutive day 
    cum_img = cum_img.where(this_img.eq(1).And(prev_img.eq(1)), cum_img.add(1));
  ## If < 35 deg, reset counter
    cum_img = cum_img.where(this_img.neq(1), 0);
  ## Last data from the time range
    max_run = max_run.where(cum_img.gt(max_run),cum_img);
  ## This return value becomes cum_prev input
    return cum_img.addBands(this_img).addBands(max_run);



def exportImage(img, geometry, folder, dscr, proj):
    task = ee.batch.Export.image.toDrive(image=img,  # an ee.Image object.
                                         region=geometry,  # an ee.Geometry object.
                                         description=dscr,
                                         folder=folder,
                                         fileNamePrefix=dscr,
                                         scale=1000,
                                         crs=proj)
    return task

def millSec(date):
    return ee.Date(date).millis()

In [6]:
def urbanBoundaries1(gdf,city,crs):
    b = 5000
    geo = gdf[gdf['UC_NM_MN']==city].reset_index(drop=True)
    bounds = geo.total_bounds
    lon_point_list = [bounds[0]-b, bounds[0]-b, bounds[2]+b, bounds[2]+b]
    lat_point_list = [bounds[1]-b, bounds[3]+b, bounds[3]+b, bounds[1]-b]
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    return geo, gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])

def returnGeometry(gdf):
    bounds = gdf.total_bounds
    return ee.Geometry.Polygon(
            [[[bounds[0], bounds[1]],
            [bounds[0], bounds[3]],
            [bounds[2], bounds[3]],
            [bounds[2], bounds[1]]]])



In [5]:
## cities in the world
dfc = gpd.read_file('../data_revision/cities/all/gdfCities.shp')
crs = dfc.crs
cities = dfc['UC_NM_MN']

In [22]:
years_str = []
for y in range(2010,2022):
    years_str.append(str(y))
    
proxy=-999

In [154]:
cols = ['idx', 'city', 't_urban', 't_rural', 'SUHI']
dfSUHI = pd.DataFrame(columns=cols)

for i,c in enumerate(cities[0:200]):
    # 1. Define boundary
    gdf_c, bounds = urbanBoundaries1(dfc,c,crs)
    
    ######## Entire urban environmnt
    bounds = bounds.to_crs("EPSG:4326")
    geometry = returnGeometry(bounds)
    
    
    ####### Urban
    geo = dfc[dfc['UC_NM_MN']==c].reset_index(drop=True)
    geo = geo.to_crs("EPSG:4326")
    g = [i for i in geo.geometry]

    all_coords = []
    if type(g[0].boundary)==shapely.geometry.linestring.LineString:
        coords = np.dstack(g[0].boundary.coords.xy).tolist()
        all_coords.append(*coords) 
        geometry2 = ee.Geometry.Polygon(all_coords)
    else:
        for b in g[0].boundary: # for first feature/row
            coords = np.dstack(b.coords.xy).tolist()
            all_coords.append(*coords) 
        geometry2 = ee.Geometry.Polygon(all_coords)
    
    ###### Rural 
    tmp = bounds['geometry'][0]
    tmp1 = geo['geometry'][0]
    tmp2 = tmp.difference(tmp1)
    gdftmp =  gpd.GeoDataFrame(index=[0], crs=bounds.crs, geometry=[tmp2])
    g2 = [i for i in gdftmp.geometry]
    
    all_coords = []
    for b in g2[0].boundary: # for first feature/row
        coords = np.dstack(b.coords.xy).tolist()
        all_coords.append(*coords) 
    geometry3 = ee.Geometry.Polygon(all_coords)
    
    
    mod11a1_day = ee.ImageCollection ('MODIS/006/MOD11A1').select(['LST_Day_1km', 'QC_Day'])\
          .map(toCelciusDay).filterBounds(geometry)

    
    #  Read temperature file
    fileT = '../data_revision/cities/air_temperature/%s/temperature_new.csv'%c
    dfT = pd.read_csv(fileT)

    # Define warmest months in 20 years (2002, 2021)
    dfT['valid_time'] = pd.to_datetime(dfT['valid_time'])
    df_filtered = dfT[dfT['valid_time'].dt.strftime('%Y').isin(years_str)].reset_index(drop=True)

    ################ get the three warmest month of the year
    df_filtered = df_filtered.groupby('valid_time')['t2m'].mean().reset_index()
    df_filtered['year'] =df_filtered['valid_time'].dt.year
    df_filtered['month'] =df_filtered['valid_time'].dt.month
    grouped = df_filtered.sort_values('t2m', ascending=False).groupby(['year'])
    result = grouped.head(3).sort_values('year').reset_index()
    #dateList = list(result.valid_time.dt.strftime('%Y-%m-%d'))
    #datelist = ee.List(dateList).map(millSec)
    dates = []
    delta = datetime.timedelta(days=1)

    dates=[]
    for vd in result.valid_time:
        year = vd.year
        month = vd.month
        my_date = datetime.date(year, month, 1)
        while my_date.month == month:
            dates.append((my_date).strftime('%Y-%m-%d'))
            my_date += delta

    datelist = ee.List(dates).map(millSec)
    
    ## Day
    img = mod11a1_day.filter(ee.Filter.calendarRange(2010,2022,'year'))
    img = img.filter(ee.Filter.inList("system:time_start", datelist)) 
    ## remove bad pixels
    LSTD_QC = img.map(QC_Day_mask);
    lst = ee.ImageCollection(LSTD_QC.select('LST_Day_1km')).mean().float();
    
    ########### Urban area
    lst2 = lst.clip(geometry2)
    lst2 = lst2.reduceRegion(**{
      'reducer': ee.Reducer.mean(),
      'geometry': geometry.getInfo(),
      'scale': 1000,
      'crs': 'EPSG:4326',
    })
    
    
    ########### Rural area
    lst3 = lst.clip(geometry3)

    lst3 = lst3.reduceRegion(**{
      'reducer': ee.Reducer.mean(),
      'geometry': geometry.getInfo(),
      'scale': 1000,
      'crs': 'EPSG:4326',
    })
    
    suhi = lst2.getInfo()['LST_Day_1km'] - lst3.getInfo()['LST_Day_1km']
    data = [i, c, lst2.getInfo()['LST_Day_1km'], lst3.getInfo()['LST_Day_1km'], suhi]
    dfSUHI = dfSUHI.append(pd.DataFrame(columns=cols, data=[data]), ignore_index=True)
    print(suhi, c)

3.8228270062784055 Guangzhou
1.3969142083623183 Los Angeles
6.338105612636468 New York
7.580474271172136 Tokyo
5.135229503652585 Jakarta
5.830747157870043 Chicago
2.7466481727946928 Dallas
4.169225919745088 Houston
3.283587149813659 Shanghai
1.169416660308169 Dhaka
6.750693079580547 Osaka [Kyoto]
5.809803386986058 Miami
0.7894427708022782 Kolkata
6.989424691155261 Nagoya
2.519012159739269 Bangkok
7.136888248667518 Detroit
0.3155910878735604 Comilla
1.9090070151469831 Delhi [New Delhi]
4.004310348479507 Jieyang
4.2341438227646115 Seoul
-0.7512261772713771 Phoenix
4.734707244920099 Beijing
7.52760445433649 Mexico City
7.495008923394362 Quezon City [Manila]
8.478089054449356 Toronto
7.505577819110883 São Paulo
3.2991130562656252 Buenos Aires
4.616865975975184 Suzhou
8.383951366776596 Seattle
4.1062055947067755 Moscow
4.199651438823622 London
-2.260416892083043 Tijuana
5.6482015245472255 Surabaya
3.298534922628537 Johannesburg
4.47793494294098 Paris
4.223416195597469 Melbourne
-1.360134663